### Aggregation & Grouping

An essential piece of analysis of large data is efficient summarization: computing aggregations like `sum(), mean(), median(), min(), and max()` in which a single number gives insight into the nature of a potentially large dataset. We'll explore aggregations in Pandas, from simple operations akin to what we've seen on NumPy arrays, to more sophisticated operations based on the concept of a `groupby`.

* Planetary dataset available from Seaborn
* provides information on planets that astronomers have discovered around other stars

In [1]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [2]:
planets.shape

(1035, 6)

In [3]:
# Numpy Refresher
import numpy as np
import pandas as pd
# Set Seed and pull 5 values to form series
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [4]:
ser.describe()

count    5.000000
mean     0.562385
std      0.308748
min      0.156019
25%      0.374540
50%      0.598658
75%      0.731994
max      0.950714
dtype: float64

In [5]:
ser.mean(), ser.std()

(0.5623850983416314, 0.30874824961862174)

* aggregates return sa single value as expected for a series (or 1-d type of dimensional value aggreagtion)

In [6]:
print(type(rng.rand(5)), rng.rand(5)) # we can pass this method call when creating a default DataFrame

<class 'numpy.ndarray'> [0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]


#### Column Aggregates for DataFrame

In [7]:
df = pd.DataFrame({'A': rng.rand(5), 'B': rng.rand(5)})
df

,A,B
0,0.183405,0.611853
1,0.304242,0.139494
2,0.524756,0.292145
3,0.431945,0.366362
4,0.291229,0.456070


In [8]:
df.mean()

A    0.347115
B    0.373185
dtype: float64

In [9]:
# Specify axis argument, for aggregation within each row .. by specifying columns
df.mean(axis='columns')

0    0.397629
1    0.221868
2    0.408451
3    0.399153
4    0.373650
dtype: float64

#### Back to Planets


In [10]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [11]:
# any difference when dropping nulls? Yes! Quite a difference (see overall count) would drop any row with a null value from dropna() call
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


* To go deeper into the data, however, simple aggregates are often not enough. 
* The `groupby` operation allows for quick and efficient aggregate computation on subsets of data

### GroupBy: Split, Apply, Combine

* The split step involves breaking up and grouping a DataFrame depending on the value of the specified key.  
* The apply step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.  
* The combine step merges the results of these operations into an output array.


In [12]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                  'data': range(6)},
                 columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [13]:
df.groupby('key')

* Notice that what is returned is not a set of DataFrames, but a `DataFrameGroupBy` object. This object is where the magic is: you can think of it as a special view of the DataFrame, which is poised to dig into the groups but does no actual computation until the aggregation is applied.

In [14]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


# The GroupBy object 

The GroupBy object is a very flexible abstraction. In many ways, you can simply treat it as if it’s a collection of `DataFrame`s, and it does the difficult things under the hood. Let’s see some examples using the Planets data. Perhaps the most important operations made available by a `GroupBy` are `aggregate, filter, transform, and apply`.


#### Column Indexing

In [15]:
# Column indexing
print(planets.columns.tolist())
planets.groupby('method'), planets.groupby('method')['orbital_period']

['method', 'number', 'orbital_period', 'mass', 'distance', 'year']


(<pandas.core.groupby.generic.DataFrameGroupBy object at 0x7fa6a97a2700>,
 <pandas.core.groupby.generic.SeriesGroupBy object at 0x7fa6a97c5250>)

* Here we've selected a particular `Series` group from the orginal `DataFrame` groupby reference to its column name.
* As with the `GroupBy` object, no computation is done until we call some aggregate on the object

In [16]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

#### Iteration over groups
`GroupBy` object supports direct iteration over the groups, returning each group as a `Series` or `DataFrame`

In [17]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


* This can be useful for doing certain things manually, though it is often much faster to use the built-in-`apply` functionality

#### Dispatch methods
Through some Python class magic, any method not explicitly implemented by the `GroupBy` object will be passed through and called on the groups, whether they are `DataFrame` or `Series` objects. For example, you can use the `describe()` method of DataFrames to perform a set of aggregations that describe each group in the data:

In [18]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


In [19]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.0
       Eclipse Timing Variations           9.0
       Imaging                            38.0
       Microlensing                       23.0
       Orbital Brightness Modulation       3.0
                                         ...  
max    Pulsar Timing                    2011.0
       Pulsation Timing Variations      2007.0
       Radial Velocity                  2014.0
       Transit                          2014.0
       Transit Timing Variations        2014.0
Length: 80, dtype: float64

* Looking at this table helps us to better understand the data: for example, the vast majority of planets have been discovered by the Radial Velocity and Transit methods, though the latter only became common (due to new, more accurate telescopes) in the last decade. The newest methods seem to be Transit Timing Variation and Orbital Brightness Modulation, which were not used to discover a new planet until 2011.
* Notice that they are applied to each individual group, and the results are then combined within GroupBy and returned. Again, any valid DataFrame/Series method can be used on the corresponding GroupBy object, which allows for some very flexible and powerful operations!

#### Aggregate, filter, transform, apply
The preceding discussion focused on aggregation for the combine operation, but there are more options available. In particular, GroupBy objects have `aggregate(), filter(), transform(), and apply()` methods that efficiently implement a variety of useful operations before combining the grouped data.



In [20]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                  'data1': range(6),
                  'data2': rng.randint(0,10,6)},
                 columns=['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Aggregation 
We’re now familiar with `GroupBy` aggregations with `sum(), median()`, and the like, but the `aggregate()` method allows for even more flexibility. It can take a string, a function, or a list thereof, and compute all the aggregates at once. Here is a quick example combining all these:


In [21]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [22]:
df.groupby('key').agg(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

#### Filtering
A `filtering` operation allows you to drop data based on the group properties. For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [23]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [24]:
def filter_func(x):
    return x['data2'].std() > 4

In [25]:
print(df.groupby('key').std())

       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641


In [26]:
print(df.groupby('key').filter(filter_func))

  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9


#### Transformation 
While aggregation must return a reduced version of the data, `transformation` can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common example is to center the data by subtracting the group-wise mean:

In [27]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


* Each key shows the value in data1 and data2 value subtracted by it's column groupby mean
    * First row Example
        * `A` key mean for `data1` column is 0 + 3 / 2 == 1.5
        * Return for row 0 and 4 (both relating to key `A`) in this column is the original value minus the mean above (this is -1.5 and 1.5 respectfully (0 - 1.5 and 3 - 1.5) 

#### Apply() method
The `apply()` method lets you apply an arbitrary function to the group results. The function should take a `DataFrame`, and return either a Pandas object (e.g., DataFrame, Series) or a scalar; the combine operation will be tailored to the type of output returned. 
For example, here is an `apply()` that normalizes the first column by the sum of the second:

In [28]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [29]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

In [30]:
print(df.groupby('key').apply(norm_by_data2))

  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


* Little detail about aboe
    * Key `B` Example
        * Row 1 above for Key B is applying as follows
            * 1) The value of 1 for B in row 1 column `data1` is divided (and then equals) the sum of the same key's column `data2` sum
            * This is equal to (1 / (0+7)) == 1 / 7 == 0.142857
        * Row 2 for Key C applys similary
            * 2 / (3 + 9) == 2/12 (1/6) == 0.166667

### Specifying the split key

In [36]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [34]:
# list, array, series, or index providing the grouping keys (I have no idea how this working - can't see how L is pairing or assigning the values )
L = [0,1,0,1,2,0]
df.groupby(L).sum()

,data1,data2
0,7,17
1,4,3
2,4,7


In [35]:
df.groupby(df['key']).sum() # this makes total sense!

,data1,data2
key,,
A,3,8
B,5,7
C,7,12


In [37]:
# dictionary or series - maps index values to the group keys
df2 = df.set_index('key')
print(df2)
mapping = {'A':'vowel', 'B':'consonant', 'C':'consonant'}
print(df2.groupby(mapping).sum())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
           data1  data2
key                    
consonant     12     19
vowel          3      8


* The above is more or less aggregating the key values that share the same value received from the mapping dictionary
    * For example, 'B' and 'C' are aggregated together for the totals in the column(s) data1 and data2 and then summed together

In [39]:
## Passing a Python Function
print(df2)
df2.groupby(str.lower).mean()

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9


,data1,data2
key,,
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0


In [40]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
key,key,,
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### Grouping Example
* Count discovered planets by method and by decade

In [49]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [47]:
decade = 10 * (planets['year'] // 10) # floor division
print(decade[:5])
# Mutate to string and concatenate with a 's'
decade = decade.astype(str) + 's'
decade.name = 'decade'
print(decade[:5])

0    2000
1    2000
2    2010
3    2000
4    2000
Name: year, dtype: int64
0    2000s
1    2000s
2    2010s
3    2000s
4    2000s
Name: decade, dtype: object


In [52]:
planets.loc[planets['method'] == 'Astrometry']

,method,number,orbital_period,mass,distance,year
113,Astrometry,1,246.36,NaN,20.77,2013
537,Astrometry,1,1016.00,NaN,14.98,2010


In [54]:
planets.loc[planets['method'] == 'Eclipse Timing Variations']

,method,number,orbital_period,mass,distance,year
32,Eclipse Timing Variations,1,10220.00,6.05,NaN,2009
37,Eclipse Timing Variations,2,5767.00,NaN,130.72,2008
38,Eclipse Timing Variations,2,3321.00,NaN,130.72,2008
39,Eclipse Timing Variations,2,5573.55,NaN,500.00,2010
40,Eclipse Timing Variations,2,2883.50,NaN,500.00,2010
41,Eclipse Timing Variations,1,2900.00,NaN,NaN,2011
42,Eclipse Timing Variations,1,4343.50,4.20,NaN,2012
43,Eclipse Timing Variations,2,5840.00,NaN,NaN,2011
44,Eclipse Timing Variations,2,1916.25,NaN,NaN,2011


In [59]:
planets.groupby('method')['number'].agg(['sum', 'count'])

,sum,count
method,,
Astrometry,2,2
Eclipse Timing Variations,15,9
Imaging,50,38
Microlensing,27,23
Orbital Brightness Modulation,5,3
Pulsar Timing,11,5
Pulsation Timing Variations,1,1
Radial Velocity,952,553
Transit,776,397


In [48]:
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


* planets dataframe groupbys initially by method then by created series decade above
    * a bit lost on how the groupby result is correctly performing the second group based on a decade string but hey that's why we love pandas black magic!

In [61]:
planets.groupby(['method', decade])['number'].agg(['count', 'sum'])

count  sum
method                        decade            
Astrometry                    2010s       2    2
Eclipse Timing Variations     2000s       3    5
                              2010s       6   10
Imaging                       2000s      20   29
                              2010s      18   21
Microlensing                  2000s      10   12
                              2010s      13   15
Orbital Brightness Modulation 2010s       3    5
Pulsar Timing                 1990s       3    9
                              2000s       1    1
                              2010s       1    1
Pulsation Timing Variations   2000s       1    1
Radial Velocity               1980s       1    1
                              1990s      28   52
                              2000s     309  475
                              2010s     215  424
Transit                       2000s      62   64
                              2010s     335  712
Transit Timing Variations     2010s       4    9